<h1>Analysis of Tumor Exome Mutations</h1>

<!--[cancer_cells](https://swami.wustl.edu/wp-content/uploads/2015/07/cancers-cells-1038x576.jpg)-->
<img src="https://swami.wustl.edu/wp-content/uploads/2015/07/cancers-cells-1038x576.jpg" alt="cancer cells" width="600px">

<h2>Author: Yogindra Raghav</h2>
<h2>April 2019</h2>

In [2]:
import pandas

<h3>Load All DataFrames into Dict</h3>

In [6]:
# dict for all MAF data 
all_mafs = {}

# for all 50 MAF files 
for i in range(0,50): 
    # load all MAF data into dict using patient number as key 
    all_mafs["Patient-"+str(i)] = pandas.read_csv("mafs/Patient-"+str(i)+".somatic.snvs.maf", sep="\t")
    

<h3>Subset for Mutations that 